In [1]:
import geopandas as gpd
import pandas as pd
import math
import folium
from folium import Marker, CircleMarker
from shapely.geometry import Point
import sys
from folium.plugins import HeatMap, MarkerCluster, FloatImage
import re

def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')




In [2]:
new_hospi = gpd.read_file('RS_Pop_Envi.shp')
new_hospi.head()

,Address,District,Lat,Long,Name,Province,Type,sum,sum_1,geometry
0,"Jl. By Pass Ngurah Rai No.100 X, Kuta - Badung",Badung,-8.677750,115.205522,RSK Bedah dan Penyakit Dalam BIMC,Bali,J,4.751959,0.476117,POINT (115.20552 -8.67775)
1,"Jl. Raden Mattaher 33,Jambi",None,-1.591744,103.618533,Rumkit Tk IV Dr. Bratanata Unang Jambi,Jambi,TNI AD,3.346735,0.861326,POINT (103.61853 -1.59174)
2,Jl. Imam Bonjol Karawaci No 38 TNG,Kota Tangerang,-6.179225,106.626283,RSAB Sari Asih,Banten,RSU,6.780755,0.252343,POINT (106.62628 -6.17922)
3,Jl. Iko Jatmika 1 Rangkasbitung,Lebak,-6.369178,106.283111,RSU Dr Adjidarmo,Banten,RS non Pendidikan,0.069562,0.633958,POINT (106.28311 -6.36918)
4,Jl. KH.HASYIM ASHARI NO.24 Tangerang,Kota Tangerang,-6.183983,106.647747,RS Usada Insani,Banten,U,6.293740,0.150169,POINT (106.64775 -6.18398)


In [3]:
# new_hospi['sum']

norm_pop, norm_envi, max_pop, max_envidx = [], [], max(new_hospi['sum']), max(new_hospi.sum_1)
for i in range(len(new_hospi)):
    norm_pop.append(new_hospi.iloc[i]['sum'] / max_pop)
    norm_envi.append(new_hospi.iloc[i].sum_1 / max_envidx)

new_hospi['Norm_Pop'] = norm_pop
new_hospi['Norm_Env'] = norm_envi
new_hospi.head()

,Address,District,Lat,Long,Name,Province,Type,sum,sum_1,geometry,Norm_Pop,Norm_Env
0,"Jl. By Pass Ngurah Rai No.100 X, Kuta - Badung",Badung,-8.677750,115.205522,RSK Bedah dan Penyakit Dalam BIMC,Bali,J,4.751959,0.476117,POINT (115.20552 -8.67775),0.409940,0.482668
1,"Jl. Raden Mattaher 33,Jambi",None,-1.591744,103.618533,Rumkit Tk IV Dr. Bratanata Unang Jambi,Jambi,TNI AD,3.346735,0.861326,POINT (103.61853 -1.59174),0.288715,0.873178
2,Jl. Imam Bonjol Karawaci No 38 TNG,Kota Tangerang,-6.179225,106.626283,RSAB Sari Asih,Banten,RSU,6.780755,0.252343,POINT (106.62628 -6.17922),0.584959,0.255815
3,Jl. Iko Jatmika 1 Rangkasbitung,Lebak,-6.369178,106.283111,RSU Dr Adjidarmo,Banten,RS non Pendidikan,0.069562,0.633958,POINT (106.28311 -6.36918),0.006001,0.642681
4,Jl. KH.HASYIM ASHARI NO.24 Tangerang,Kota Tangerang,-6.183983,106.647747,RS Usada Insani,Banten,U,6.293740,0.150169,POINT (106.64775 -6.18398),0.542945,0.152236


In [5]:
m = folium.Map(location=[0.7893,113.9213], zoom_start=5)

mc = MarkerCluster()

for idx, row in new_hospi.iterrows():
    if not math.isnan(row['Long']) and not math.isnan(row['Lat']):
        mc.add_child(folium.Marker([row['Lat'], row['Long']], 
                                   tooltip='This is a tooltip with <br> multiple lines')).add_to(m)

# folium.LatLngPopup().add_to(mc)

embed_map(m, 'hospital.html')

In [6]:
def popcolormap(pop):
    if pop <= 1.1:
        return '#F3F3F3'
    elif pop > 1.1 and pop <= 3.5:
        return '#EBC5DE'
    else:
        return '#E6A3CF'
    
def popcolormap_v2(pop):
    if pop <= 1.1:
        return '#EBC5DE'
    elif pop > 1.1 and pop <= 3.5:
        return '#E6A3CF'
    else:
        return '#E171BB'

def envcolormap(env):
    if env <= 0.35:
        return '#F3F3F3'
    elif env > 0.35 and env <= 0.74:
        return '#C2F1CD'
    else:
        return '#8BE2AF'

def envcolormap_v2(env):
    if env <= 0.35:
        return '#C2F1CD'
    elif env > 0.35 and env <= 0.74:
        return '#8BE2AF'
    else:
        return '#50E18C'

def combcolormap(pop, env):
    if pop <= 1.1 and env <= 0.35:
        return '#F3F3F3'
    elif (pop > 1.1 and pop <= 3.5) and  env <= 0.35:
        return '#EBC5DE'
    elif pop > 3.5 and env < 0.35:
        return '#E6A3CF'
    elif pop <= 1.1 and (env > 0.35 and env <= 0.74):
        return '#C2F1CD'
    elif pop <= 1.1 and env > 0.74:
        return '#8BE2AF'
    elif (pop > 1.1 and pop <= 3.5) and (env > 0.35 and env <= 0.74):
        return '#9FC7D3'
    elif (pop > 1.1 and pop <= 3.5) and env > 0.74:
        return '7EC6B0'
    elif pop > 3.5 and (env > 0.35 and env <= 0.74):
        return '#BC9FCF'
    elif pop > 3.5 and env > 0.74:
        return '#7B8EAF'
    
def combcolormap_v2(pop, env):
    if pop <= 1.1 and env <= 0.35:
        return '#F3F3F3'
    elif (pop > 1.1 and pop <= 3.5) and  env <= 0.35:
        return '#E6A3CF'
    elif pop > 3.5 and env < 0.35:
        return '#E171BB'
    elif pop <= 1.1 and (env > 0.35 and env <= 0.74):
        return '#8BE2AF'
    elif pop <= 1.1 and env > 0.74:
        return '#50E18C'
    elif (pop > 1.1 and pop <= 3.5) and (env > 0.35 and env <= 0.74):
        return '#76C3DA'
    elif (pop > 1.1 and pop <= 3.5) and env > 0.74:
        return '#46C9A1'
    elif pop > 3.5 and (env > 0.35 and env <= 0.74):
        return '#AD6DD6'
    elif pop > 3.5 and env > 0.74:
        return '#5579B6'
    

In [7]:
# Map for pop layer
m1 = folium.Map(location=[0.7893,113.9213], zoom_start=5, tiles='CartoDB dark_matter')
pop_leg = 'Pop.png'

for i in range(len(new_hospi)):
    # print(i)
    folium.Circle(
        location=[new_hospi.iloc[i].geometry.y, new_hospi.iloc[i].geometry.x],
        radius=5000,
        fill=True,
        fill_color=popcolormap_v2(new_hospi.iloc[i]['sum']),
        weight = 0,
    ).add_to(m1)


FloatImage(pop_leg, bottom=5, left=3).add_to(m1)

embed_map(m1, 'pop.html')

In [8]:
# Map for env layer
m2 = folium.Map(location=[0.7893,113.9213], zoom_start=5, tiles='CartoDB dark_matter')
env_leg = 'Env.png'

for i in range(len(new_hospi)):
    # print(i)
    folium.Circle(
        location=[new_hospi.iloc[i].geometry.y, new_hospi.iloc[i].geometry.x],
        radius=5000,
        fill=True,
        fill_color=envcolormap_v2(new_hospi.iloc[i].sum_1),
        weight = 0,
    ).add_to(m2)


FloatImage(env_leg, bottom=5, left=3).add_to(m2)

embed_map(m2, 'env.html')

In [19]:
# Map for combine layer
m3 = folium.Map(location=[0.7893,113.9213], zoom_start=5, tiles='CartoDB dark_matter')
comb_leg = 'Combine 1.png'

for i in range(len(new_hospi)):
    # print(i)
    folium.Circle(
        location=[new_hospi.iloc[i].geometry.y, new_hospi.iloc[i].geometry.x],
        radius=5000,
        fill=True,
        fill_color=combcolormap_v2(new_hospi.iloc[i]['sum'],new_hospi.iloc[i].sum_1),
        weight = 0,
    ).add_to(m3)


FloatImage(comb_leg, bottom=5, left=3).add_to(m3)

embed_map(m3, 'comb.html')

In [17]:
# comb with marker
m4 = folium.Map(location=[0.7893,113.9213], zoom_start=5, tiles='CartoDB dark_matter')

mc = MarkerCluster()

for idx, row in new_hospi.iterrows():
    if not math.isnan(row['Long']) and not math.isnan(row['Lat']):
        mc.add_child(folium.Marker([row['Lat'], row['Long']], 
                                   tooltip='{} <br> Population Vulnerability : {}<br> Mosquito Resistance : {}'.format(row['Name'], row['sum'], row['sum']))).add_to(m4)


comb_leg = 'Combine 1.png'

for i in range(len(new_hospi)):
    # print(i)
    folium.Circle(
        location=[new_hospi.iloc[i].geometry.y, new_hospi.iloc[i].geometry.x],
        radius=5000,
        fill=True,
        fill_color=combcolormap_v2(new_hospi.iloc[i]['sum'],new_hospi.iloc[i].sum_1),
        weight = 0,
    ).add_to(m4)


FloatImage(comb_leg, bottom=5, left=3).add_to(m4)
# folium.LatLngPopup().add_to(mc)

embed_map(m4, 'hospital_comb.html')

In [16]:
# Pop with marker
m5 = folium.Map(location=[0.7893,113.9213], zoom_start=5, tiles='CartoDB dark_matter')

mc = MarkerCluster()

for idx, row in new_hospi.iterrows():
    if not math.isnan(row['Long']) and not math.isnan(row['Lat']):
        mc.add_child(folium.Marker([row['Lat'], row['Long']], 
                                   tooltip='{} <br> Population Vulnerability : {}'.format(row['Name'], row['sum']))).add_to(m5)


pop_leg = 'Pop.png'

for i in range(len(new_hospi)):
    # print(i)
    folium.Circle(
        location=[new_hospi.iloc[i].geometry.y, new_hospi.iloc[i].geometry.x],
        radius=5000,
        fill=True,
        fill_color=popcolormap_v2(new_hospi.iloc[i]['sum']),
        weight = 0,
    ).add_to(m5)

FloatImage(pop_leg, bottom=5, left=3).add_to(m5)
# folium.LatLngPopup().add_to(mc)

embed_map(m5, 'hospital_pop.html')

In [14]:
# env with marker
m6 = folium.Map(location=[0.7893,113.9213], zoom_start=5, tiles='CartoDB dark_matter')

mc = MarkerCluster()

for idx, row in new_hospi.iterrows():
    if not math.isnan(row['Long']) and not math.isnan(row['Lat']):
        mc.add_child(folium.Marker([row['Lat'], row['Long']], 
                                   tooltip='{} <br> Mosquito Resistance : {}'.format(row['Name'], row['sum_1']))).add_to(m6)


env_leg = 'Env.png'

for i in range(len(new_hospi)):
    # print(i)
    folium.Circle(
        location=[new_hospi.iloc[i].geometry.y, new_hospi.iloc[i].geometry.x],
        radius=5000,
        fill=True,
        fill_color=envcolormap_v2(new_hospi.iloc[i].sum_1),
        weight = 0,
    ).add_to(m6)

FloatImage(env_leg, bottom=5, left=3).add_to(m6)
# folium.LatLngPopup().add_to(mc)

embed_map(m6, 'hospital_env.html')